In [12]:
#import standard libraries
import os
from typing import List
import json
import logging 

In [1]:
#import Langchain Lbraries
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_experimental.text_splitter import SemanticChunker
from langchain.tools import tool
from langchain.agents import create_react_agent
from langsmith import Client
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain. retrievers.multi_query import MultiQueryRetriever
from langchain_core.messages.human import HumanMessage
from langchain.memory import ConversationBufferMemory



c:\Windows\System32\Knowledgenow\env\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [33]:
#import pinecone libraries
import pinecone
from langchain_pinecone import PineconeVectorStore

In [34]:
#import utilis
from airtable import AirTable

In [35]:
#import secret keys
from dotenv import load_dotenv 
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pine_cone_api = os.getenv("PINECONE_API_KEY")

In [36]:
#setup Langsmith for debugging
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__c4c7a9772ef046038636017b03b5f17b"

In [37]:
Langsmith_client = Client()

In [66]:
from importlib import reload
import prompts.prompts_2024_03_26 as prompts
reload(prompts)



<module 'prompts.prompts_2024_03_26' from 'c:\\Windows\\System32\\Knowledgenow\\prompts\\prompts_2024_03_26.py'>

In [91]:
airtable = AirTable()
airtable.post_data_to_table(data={"name_of_dev": "",
                "model_name": "",
                "human_prompt": "",
                "global_context": "",
                "context_entrichment_agent_system_prompt": "",
                "enriched_prompt_output": "",
                "rag_extractor_agent_system_prompt": "",
                "rag_optimized_queries_output": "",
                "total_chunks_retrieved": "",
                "chunk_size": "",
                "knowledge_base_docs_list": "",
                "evaluator_agent_system_prompt": "",
                "reranked_rag_queries_output": "",
                "answer_agent_system_prompt": "",
                "final_answer_to_user": ""})



        To post data to the Airtable table.
        Args:
            data (dict): A dictionary with the format `{'column_name': data_to_post, ...}`.
        
        Keys for dictionary:
              {
                name_of_dev: "",
                model_name: "",
                human_prompt: "",
                global_context: "",
                context_entrichment_agent_system_prompt: "",
                enriched_prompt_output: "",
                rag_extractor_agent_system_prompt: "",
                rag_optimized_queries_output: "",
                total_chunks_retrieved: "",
                chunk_size: "",
                knowledge_base_docs_list: "",
                evaluator_agent_system_prompt: "",
                reranked_rag_queries_output: "",
                answer_agent_system_prompt: "",
                final_answer_to_user: ""
              }
        
data posted to Airtable successfully.


Context Agent 

In [3]:
#import global context file 
with open("Fastrise_global_context.txt", "r") as file:
    global_context = file.read()

In [29]:
@tool
def Contextawareness(query:str)-> str:
    "use this tool any time you get question about meetings, tasks, activities in a organization called Fastrise"
    global global_context
    return global_context

In [30]:
tools = [Contextawareness]

In [31]:
#create context_enrichment prompt template

context_enrichment_template = prompts.context_enrichment_template

context_prompt = PromptTemplate.from_template(template=context_enrichment_template, MessagePlaceholder=["(agent_scratchpad)"])



In [38]:
#initiaze language models
llm  = ChatOpenAI(
    openai_api_key=openai_api_key, 
    model="gpt-4-turbo-preview", 
    temperature= 0.0
    )

In [39]:

context_agent = create_react_agent(llm=llm, 
                           tools=tools, 
                           prompt=context_prompt)

In [40]:
context_agent_executor = AgentExecutor(
    agent=context_agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors=True, 
    )

In [41]:
input = "what did alejo and bartek discuss in the meeting"

context_enriched_output = context_agent_executor.invoke({"input": input})



> Entering new AgentExecutor chain...
Question: what did Alejo and Bartek discuss in the meeting
Thought: To enrich this prompt, I need to understand the context of Alejo and Bartek's roles within Fasterise, the projects they are involved in, and any recent organizational goals or challenges that might influence the topics of their discussion.
Action: Contextawareness
Action Input: Alejo and Bartek meeting discussion topics### Mission and Vision
- *Mission and Vision*: Fasterise is centered around leveraging AI technology to amplify human potential, prioritizing ethical AI development that enriches human capabilities. The company envisions a future where AI solutions address societal needs and business challenges while augmenting the human potential and keeping it at the center of the system.
### Team Structure and Roles
- *Henry*: Brings AI and entrepreneurial insights, contributing significantly to the team's innovative capabilities in AI and LangChain.
- *Zach*: Offers sales exper

In [42]:
Enriched_prompt = context_enriched_output["output"]
print(Enriched_prompt)

In the context of Fasterise's mission to leverage AI technology to amplify human potential and its commitment to ethical AI development, Alejo and Bartek's meeting likely focused on the integration of AI technologies within ongoing projects such as KnowledgeNow, Personalized Outreach System, or the Zoom Transcriber to Pinecone Integration. Discussions may have covered operational strategies to enhance project efficiency, technical challenges in current developments, and aligning these projects with Fasterise's overarching goals.


RAG EXTRACTOR AGENT


In [43]:
class sub_query(BaseModel):

    sub_queries: list = Field(description="""The sub queries extracted from the context_enriched_prompt""")


parser = JsonOutputParser(pydantic_object=sub_query)

In [44]:
@tool
def context_enriched(rag_query:str)-> str:
    "use this tool any time you need to access context-enriched prompt for creating the sub-queries "
    global Enriched_prompt
    return Enriched_prompt

In [45]:
tools = [Contextawareness]+[context_enriched]

In [49]:
#creating a prompt template

rag_template = prompts.rag_template
rag_prompt = PromptTemplate.from_template(template=rag_template, MessagePlaceholder=["(agent_scratchpad)"])



In [50]:

Rag_agent = create_react_agent(llm=llm, tools=tools, prompt=rag_prompt)

In [51]:
Rag_agent_executor = AgentExecutor(agent=Rag_agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [52]:

Rag_sub_queries = Rag_agent_executor.invoke({"input":input ,
                       "chat_history": [], 
                       "format_instructions":parser.get_format_instructions()})
print(Rag_sub_queries)








> Entering new AgentExecutor chain...


```json
{
  "sub_queries": [
    "List all topics discussed by Alejo and Bartek in the most recent meeting.",
    "Extract any action items assigned to Alejo and Bartek from their last meeting.",
    "Identify any projects or tasks Alejo and Bartek are collaborating on as mentioned in their last meeting.",
    "Summarize the decisions made by Alejo and Bartek in their recent meeting."
  ]
}
```Invalid Format: Missing 'Action:' after 'Thought:```json
{
  "sub_queries": [
    "List all topics discussed by Alejo and Bartek in the most recent meeting.",
    "Extract any action items assigned to Alejo and Bartek from their last meeting.",
    "Identify any projects or tasks Alejo and Bartek are collaborating on as mentioned in their last meeting.",
    "Summarize the decisions made by Alejo and Bartek in their recent meeting."
  ]
}
```Invalid Format: Missing 'Action:' after 'Thought:```json
{
  "sub_queries": [
    "List all topics discussed by Alejo and Bartek in the most recent meeting."

In [65]:
Rag_query= Rag_sub_queries["output"]

# Remove the markdown code block syntax (```json and ```) from the string
Rag_query_cleaned = Rag_query.replace('```json\n', '').replace('\n```', '')

# Convert the JSON string into a dictionary
Rag_query_dict = json.loads(Rag_query_cleaned)

# Extract the sub-queries
sub_queries = Rag_query_dict["sub_queries"]

# Now you can access each sub-query by its index
sub_query_1 = sub_queries[0]
sub_query_2 = sub_queries[1]
sub_query_3 = sub_queries[2]
sub_query_4 = sub_queries[3]

# Display the extracted sub-queries
print(sub_query_1)
print(sub_query_2)
print(sub_query_3)
print(sub_query_4)



List all topics discussed by Alejo and Bartek in the most recent meeting.
Extract any action items assigned to Alejo and Bartek from their last meeting.
Identify any projects or tasks Alejo and Bartek are collaborating on as mentioned in their last meeting.
Summarize the decisions made by Alejo and Bartek in their recent meeting.


MULTIQUERY

In [34]:
#loading company docs
def load_file(data):
    loader = DirectoryLoader("data",
                    glob="**/*.txt",
                    loader_cls= TextLoader)
    text_documents = loader.load()
    return text_documents
    

 #extract the data 
extracted_data = load_file(["2024-03-12_Dev_Meeting_-_Knowledge_Now_summary_1.txt, 2024-03-12 Dev Meeting - Knowledge Now Final Draft 1cleaned.txt"])
print(extracted_data)


In [36]:
#split documents into chunks
def text_split(extracted_data):
    text_splitter = SemanticChunker(OpenAIEmbeddings(), breakpoint_threshold_type="percentile")
    documents = text_splitter.split_documents(extracted_data)

    return documents

text_chunk = text_split(extracted_data)
print("The length of the chunk:", len(text_chunk))

In [54]:
#creating embeddings

model_name = "text-embedding-3-large"
def data_embedding():
    embeddings = OpenAIEmbeddings(model=model_name)
    return embeddings

embeddings = data_embedding()

In [41]:
#initializing Vectorstore
Pinecone_api_key = os.getenv("PINECONE_API_KEY")
Your_env = "gcp-starter"
index_name = "knowledgenow"

In [43]:
#Adding embeddings into the vectorstore
vectorstore = PineconeVectorStore.from_documents(text_chunk, embeddings, index_name=index_name)

In [55]:
#initializing retrival
retriver = vectorstore.as_retriever(search_kwargs={'k':25})
multi_llm = ChatOpenAI(temperature= 0.5)
#Adding mutli query retriver
retriever = MultiQueryRetriever.from_llm(
    retriever=retriver, llm=multi_llm
)
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

NameError: name 'vectorstore' is not defined

In [ ]:
#retrieval chain
qa = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever
)


KNOWLEDGEBASE QUERY AGENT

In [65]:
#creating a custom tool
@tool
def Knowledgebase(query:str)-> str:
    """
    Use this tool when you want extract information from the Knowledgenow database"""
    result_str = qa.invoke(sub_query)
    return result_str



In [66]:
tools = [Knowledgebase]

In [59]:
#Knowledgebase_system prompt

Knowledgebase_template = prompts.Knowledgebase_template

Knowledgebase_prompt = PromptTemplate.from_template(template=Knowledgebase_template, MessagePlaceholder=["(agent_scratchpad)"])


In [60]:
Knowledgebase_agent = create_react_agent(llm=llm, tools=tools, prompt=Knowledgebase_prompt)

In [61]:
Knowledgebase_agent_executor = AgentExecutor(agent=Knowledgebase_agent, tools=tools, verbose=True,handle_parsing_errors=True, max_iterations=8)

In [64]:
Knowledgebase_agent_output= Knowledgebase_agent_executor.invoke({"sub_query": sub_query_1})

NameError: name 'sub_query_1' is not defined

In [83]:
Knowledgebase_agent_output_2= Knowledgebase_agent_executor.invoke({"sub_query": sub_query_2})



> Entering new AgentExecutor chain...
I need to query the KnowledgeNow database to find information about any action items that were assigned to Alejo and Bartek during the meeting.

Action: Knowledgebase
Action Input: "What action items were assigned to Alejo and Bartek during the meeting?"


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What were the main topics covered by Alejo and Bartek during their meeting?', '2. Can you provide a summary of the topics that Alejo and Bartek talked about in their meeting?', '3. What subjects were on the agenda for the meeting between Alejo and Bartek?']


{'question': 'List all topics discussed by Alejo and Bartek in the meeting.', 'answer': 'Alejo and Bartek discussed several topics during their meeting, including:\n\n1. Technical issues and audio problems at the start of the meeting.\n2. The outcome of a development meeting related to a specific system they are working on, including ensuring clarity on tasks, brainstorming ideas, potentially using a bot to measure messages, and the importance of each team member understanding their tasks for success.\n3. A workshop for educators and youth workers, training for facilitation, public speaking skills, and a five-day youth program.\n4. Opportunities for speaking at events about AI directed at high school graduates interested in AI.\n5. The concept of balance in our lives, especially concerning technology, and finding technology-free spaces for programs.\n6. The development of a deck for clients named Pablo and coding inspiration.\n7. The communication framework for PFA, exploring crew AI, 

In [85]:
Knowledgebase_agent_output_3= Knowledgebase_agent_executor.invoke({"sub_query": sub_query_3})



> Entering new AgentExecutor chain...
Thought: To answer this query, I need to search the KnowledgeNow database for documents or records of meetings where Alejo and Bartek made decisions. I will focus on extracting specific decisions attributed to these individuals.

Action: Knowledgebase

Action Input: Decisions made by Alejo and Bartek in the meeting


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What were the main topics covered by Alejo and Bartek during the meeting?', '2. Can you provide a summary of the topics that Alejo and Bartek talked about in their meeting?', '3. What were the key discussion points between Alejo and Bartek in the meeting?']


{'question': 'List all topics discussed by Alejo and Bartek in the meeting.', 'answer': "Alejo and Bartek discussed several topics in their meeting, including:\n\n1. The outcome of a development meeting related to a specific system they are working on, focusing on ensuring clarity on tasks, brainstorming ideas, potentially using a bot to measure messages, and emphasizing the importance of each team member understanding their tasks within the system for success.\n2. A one-day workshop for educators and youth workers, training for facilitation, public speaking skills, learning how to deliver the curriculum, and a five-day youth program with middle school students.\n3. Their relationship with technology and finding balance in our lives with technology playing a significant role. They also discussed the possibility of hosting workshops on building a project with Chat GBT.\n4. The real estate project, including project distribution, leadership team setup, marketing efforts, and profit shari

In [87]:
Knowledgebase_agent_output_4= Knowledgebase_agent_executor.invoke({"sub_query": sub_query_4})



> Entering new AgentExecutor chain...
I need to find the summary of key points Alejo and Bartek agreed upon during their meeting from the KnowledgeNow database.

Action: Knowledgebase

Action Input: Summarize the key points Alejo and Bartek agreed upon during the meeting.


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What were the main topics covered by Alejo and Bartek during the meeting?', '2. Can you provide a summary of the topics that Alejo and Bartek talked about in the meeting?', '3. What were the key discussion points between Alejo and Bartek in their meeting?']


{'question': 'List all topics discussed by Alejo and Bartek in the meeting.', 'answer': 'Alejo and Bartek discussed several topics in their meeting, including:\n\n1. The outcome of a development meeting related to a specific system they are working on, focusing on task clarity, brainstorming ideas, potentially using a bot to measure messages, and ensuring each team member understands their tasks for success.\n2. Public speaking skills and delivering curriculum, as part of a workshop for educators and youth workers, and a five-day youth program.\n3. Their relationship with technology and finding balance in life, especially considering technology\'s significant role.\n4. The possibility of hosting workshops on building a project with Chat GPT, seeking donations for tech education tools like iPads or computers.\n5. Real estate project details, including project distribution, leadership team setup, and profit sharing.\n6. The development and discussion of a communication framework and the 

In [63]:
Knowledgebase_answer_1 = Knowledgebase_agent_output["output"]
Knowledgebase_answer_2 = Knowledgebase_agent_output_2["output"]
Knowledgebase_answer_3= Knowledgebase_agent_output_3["output"]
Knowledgebase_answer_4= Knowledgebase_agent_output_4["output"]


NameError: name 'Knowledgebase_agent_output' is not defined

In [62]:
combined_output_list = [Knowledgebase_answer_1, Knowledgebase_answer_2, Knowledgebase_answer_3, Knowledgebase_answer_4]
print(combined_output_list)


NameError: name 'Knowledgebase_answer_1' is not defined

Evaluator Agent

In [112]:
#creating a custom tool
@tool
def Vectormatches(query:str)-> str:
    """
    Use this tool when you want to return vector match results for the vectordatabase"""
    global combined_output_list
    return combined_output_list

In [122]:
evaluator_template = prompts.evaluator_template

evaluator_prompt = PromptTemplate.from_template(template=evaluator_template, MessagePlaceholder=["(agent_scratchpad)"])

In [123]:

evaluator_agent = create_react_agent(llm=llm, tools=tools, prompt=evaluator_prompt)

In [124]:
evaluator_agent_executor = AgentExecutor(agent=evaluator_agent, tools=tools, verbose=True,handle_parsing_errors=True, max_iterations=8)

In [125]:
Evalutor_output=evaluator_agent_executor.invoke({
        "input":input, 
        "global_context": global_context, 
        "enriched_prompt": Enriched_prompt,
        "Vector_matches": combined_output_list})



> Entering new AgentExecutor chain...
Given the detailed vector matches and the enriched prompt, it's clear that Alejo and Bartek's meeting was multifaceted, covering a broad range of topics that align with Fasterise's strategic goals and operational projects. The discussions spanned from system development specifics, the KnowledgeNow project, real estate initiatives, to broader themes like technology's role in life and education. These discussions reflect the intersection of their expertise in operations, sales, development, and engineering, resonating with the company's mission to leverage AI technology to amplify human potential and its commitment to ethical AI development.

To synthesize a comprehensive narrative, I'll integrate the key points from the vector matches with the global context of Fasterise, focusing on how these discussions contribute to the company's projects like KnowledgeNow and the Personalized Outreach System, and how they embody the company's values of innovat

In [126]:
Evaluator_agent_output=Evalutor_output["output"]
print(Evaluator_agent_output)

Alejo and Bartek's meeting covered a wide array of topics, central to which were the development and implementation of AI technologies within Fasterise's projects like KnowledgeNow and the Personalized Outreach System. They discussed system development specifics, the role of evaluator agents, the concept of "Crew AI," and the integration of various agents to enhance operational efficiency and team collaboration. Additionally, they explored initiatives aimed at societal impact, including educational workshops for educators and youth workers, and strategic discussions on the real estate project, emphasizing leadership, profit sharing, and marketing. These discussions underscore their collaborative efforts to align with Fasterise's mission of leveraging AI to amplify human potential, guided by ethical AI development and a human-centered approach.


ANSWER AGENT

In [129]:
#creating a custom tool
@tool
def Evaluator_result(query:str)-> str:
    """
    Use this tool anytime you need to return  Re-ranked and evaluated data results from the Evaluator assistant """
    global Evaluator_agent_output
    return Evaluator_agent_output

In [136]:
Answer_template = prompts.Answer_template

Answer_prompt = PromptTemplate.from_template(template=Answer_template, MessagePlaceholder=["(agent_scratchpad)"])

In [137]:
Answer_agent = create_react_agent(llm=llm, tools=tools, prompt=Answer_prompt)

In [138]:
Answer_agent_executor = AgentExecutor(agent=Answer_agent, tools=tools, verbose=True,handle_parsing_errors=True, max_iterations=8)

In [139]:
Answer_output=Answer_agent_executor.invoke({
        "input":input, 
        "global_context": global_context, 
        "enriched_prompt": Enriched_prompt})



> Entering new AgentExecutor chain...
Given the enriched context surrounding Fasterise's mission and the specific roles of Alejo and Bartek within the company, it's clear that their meeting would have revolved around critical aspects of ongoing and future projects, particularly those that intersect their areas of expertise. Here's a synthesized overview of what Alejo and Bartek likely discussed, based on the available information:

### Meeting Discussion Points between Alejo and Bartek

1. **Integration of AI Technologies**: They likely explored strategies for incorporating RAG, LLMs, and Multi-Agent Systems into Fasterise's current projects like KnowledgeNow and the Personalized Outreach System. This discussion would align with the company's mission to leverage AI for amplifying human potential and its commitment to ethical AI development.

2. **Operational Efficiency and Customer Experience**: Given Alejo's focus on operations, sales, and development, and Bartek's expertise in engi

In [141]:
Answer_agent_output = Answer_output["output"]
print(Answer_agent_output)

Meeting Discussion Points between Alejo and Bartek at Fasterise

1. **Integration of AI Technologies**
   - **Objective**: Strategize the incorporation of RAG, LLMs, and Multi-Agent Systems into ongoing projects like KnowledgeNow and the Personalized Outreach System.
   - **Alignment**: This aligns with Fasterise's mission to leverage AI technology to amplify human potential while adhering to ethical AI development principles.

2. **Operational Efficiency and Customer Experience**
   - **Objective**: Identify ways to enhance operational efficiency and customer experience through AI technologies.
   - **Approach**: Leveraging Alejo's expertise in operations, sales, and development alongside Bartek's engineering and system design skills.

3. **Project Implementation Challenges**
   - **Objective**: Tackle potential challenges in implementing AI technologies within Fasterise's infrastructure.
   - **Focus**: Ensuring implementations align with the company's ethical standards and enhance r

In [142]:

#conversational memory 
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True

)
conversational_memory.chat_memory.messages
conversational_memory.chat_memory.add_user_message(input)
conversational_memory.chat_memory.add_ai_message(Answer_agent_output)

In [145]:

def memory2str(memory:ConversationBufferWindowMemory):
    messages = memory.chat_memory.messages
    memory_list = [
        f"Human: {mem.content}" for mem in messages 
    ]
    memory_str = "/n".join(memory_list)
    return memory_str

In [152]:
def chat(text:str):
    Answer_agent_executor.invoke({
        "input":input, 
        "global_context": global_context, 
        "enriched_prompt": Enriched_prompt})
    conversational_memory.chat_memory.add_user_message(text)
    conversational_memory.chat_memory.add_ai_message(Answer_agent_output)
    return Answer_agent_output

In [153]:
chat({"input":"what are the things they said about the project knowledgenow"})



> Entering new AgentExecutor chain...
Given the enriched context and the focus areas of Alejo and Bartek at Fasterise, it's reasonable to infer that their meeting discussion revolved around the strategic integration of AI technologies within Fasterise's current and upcoming projects. Specifically, they likely deliberated on how to effectively apply RAG, LLMs, and Multi-Agent Systems to enhance the KnowledgeNow and Personalized Outreach System projects. This aligns with Fasterise's overarching mission of leveraging AI to amplify human potential while adhering to ethical AI development principles. Their conversation would have been a blend of technical planning, operational strategy, and possibly, the exploration of new AI applications that could further Fasterise's goals of operational efficiency, improved customer experience, and societal impact through AI innovation.

### Key Discussion Points:
- **Integration of AI Technologies**: Alejo and Bartek likely discussed the technical asp

: 